In [1]:
import sys
sys.path.append("../")

In [18]:
import torch
from torch import nn
import torch.nn.functional as F

In [130]:
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data, DataLoader

In [20]:
import numpy as np

In [77]:
points = np.random.rand(10, 2)
points = torch.from_numpy(points).to(dtype=torch.float)

In [78]:
edges = torch.zeros((2,0), dtype=torch.long)

In [79]:
data = Data(x=points, edge_index=edges)

In [80]:
data

Data(edge_index=[2, 0], x=[10, 2])

In [164]:
def generate_batch(n=10, n_graphs=8):
    graphs = []
    for _ in range(n_graphs):
        
        points = np.random.rand(n, 2)
        label = 0
        
        if np.random.rand() > 0.5:
            points = points * 5
            label = 1

        points = torch.from_numpy(points).to(dtype=torch.float)
        edges = torch.zeros((2,0), dtype=torch.long)
        
        graph = Data(x=points, edge_index=edges, y=label)
        graphs.append(graph)
        
    return graphs

In [165]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GCNConv(2, 16)
        self.conv2 = GCNConv(16, 2)
        self.fc = nn.Linear(10*2, 1)

    def forward(self, data):
        print(data)
        x, edge_index = data.x, data.edge_index

        print(x.shape)
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        print(x.shape)
        x = self.fc(x)

        return torch.sigmoid(x)

In [167]:
model = Net()

In [155]:
model(data)

tensor([0.4624], grad_fn=<SigmoidBackward>)

In [156]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

In [157]:
graphs = generate_batch(n_graphs=1024)
loader = DataLoader(graphs, batch_size=32)

In [169]:
for epoch in range(100):
    for batch in loader:
        optimizer.zero_grad()
        out = model(batch)
        print(out.shape)
        print(batch.y.shape)
        loss = F.binary_cross_entropy(out, batch.y)
        loss.backward()
        optimizer.step()

Batch(batch=[320], edge_index=[2, 0], x=[320, 2], y=[32])
torch.Size([320, 2])
torch.Size([320, 2])


RuntimeError: size mismatch, m1: [320 x 2], m2: [20 x 1] at /Users/soumith/b101_2/2019_02_08/wheel_build_dirs/wheel_3.7/pytorch/aten/src/TH/generic/THTensorMath.cpp:940